In [206]:
from tqdm import tqdm_notebook as tqdm 
import numpy as np
from collections import deque, Counter, defaultdict
import timeit

In [79]:
blocktrace = np.random.randint(10000, size=(100000))

In [135]:
def FIFO(blocktrace, frame):
    
    cache = deque(maxlen=frame)
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, leave=False):
        if block in cache:
            hit += 1
        
        else:
            cache.append(block)
            miss += 1
            
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    
    hitrate = hit / (hit+miss)
    return hitrate 

In [136]:
FIFO(blocktrace, 50)

0.00503

In [137]:
def LIFO(blocktrace, frame):
    
    cache = deque(maxlen=frame)
    hit, miss = 0, 0
    
    for block in tqdm(blocktrace, leave=False):
        if block in cache:
            hit += 1
            
        elif len(cache) < frame:
            cache.append(block)
            miss += 1
        
        else:
            cache.pop()
            cache.append(block)
            miss += 1
            
    hitrate = hit / (hit + miss)
    return hitrate

In [138]:
LIFO(blocktrace, 50)

0.00475

In [149]:
def LRU(blocktrace, frame):
    
    cache = deque(maxlen=frame)
    recency = deque()
    hit, miss = 0, 0
    
    for block in tqdm(blocktrace, leave=False):
        
        if block in cache:
            recency.remove(block)
            recency.append(block)
            hit += 1
            
        elif len(cache) < frame:
            cache.append(block)
            recency.append(block)
            miss += 1
            
        else:
            cache.remove(recency[0])
            recency.popleft()
            cache.append(block)
            recency.append(block)
            miss += 1
    
    hitrate = hit / (hit + miss)
    return hitrate

In [159]:
LRU(blocktrace, 500)

0.05027

In [263]:
def LFU(blocktrace, frame):
    cache = set()
    FU = defaultdict(int)
    FU_cache = defaultdict(int)

    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, desc="LFU", leave=False):
        FU[block] += 1

        if block in cache:
            hit += 1
            FU_cache[block] = FU[block]
        
        elif len(cache) < frame:
            FU_cache[block] = FU[block]
            cache.add(block)
            miss += 1

        else:
            e, f = min(FU_cache.items(), key=lambda a: a[1])
            FU_cache.pop(e)
            cache.remove(e)
            cache.add(block)
            FU_cache[block] = FU[block]
            miss += 1
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    #print("\n\nhitrate: ", hitrate)
    return hitrate

In [281]:
LFU(blocktrace, 500)

0.0504

In [346]:
'''
get the furthest accessed block. Scans OPT dictionary and selects the maximum positioned element
'''

def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            #print ( "Not Acccessing block anymore " + str(cached_block))
            return cached_block            
    for cached_block in C:
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    #print ( "chose to evict " + str(maxAccessBlock) + " since its position of access is " + str(maxAccessPosition))
    return maxAccessBlock

def belady_opt(blocktrace, frame):
    OPT = defaultdict(deque)

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index", leave=False)):
        OPT[block].append(i)    

    #print ("created OPT dictionary")    

    hit, miss = 0, 0

    C = set()
    seq_number = 0
    for block in tqdm(blocktrace, desc="OPT", leave=False):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block].popleft()
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                C.remove(fblock)
            C.add(block)
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block].popleft()

    #print ("hit count" + str(hit_count))
    #print ("miss count" + str(miss_count))
    hitrate = hit / (hit + miss)
    #print(hitrate)
    return hitrate

In [347]:
belady_opt(blocktrace, 500)

0.29049